In [1]:
import pandas as pd
df = pd.read_excel("C:/Users/user/Desktop/통합 문서1.xlsx")
df.head(3)

,단속장소명,구청구분,유예시간,유예시간X
0,BYC 주변,완산구,1.0,108
1,CGV고사점,완산구,NaN,4271
2,CGV고사점CCTV,완산구,NaN,5473


In [2]:
df1 = df.drop(columns=['구청구분', '유예시간'], axis=1)
df2 = df.drop(columns=['구청구분', '유예시간X'], axis=1)
df2.head(3)

,단속장소명,유예시간
0,BYC 주변,1.0
1,CGV고사점,NaN
2,CGV고사점CCTV,NaN


In [4]:
df2.isnull().sum()

단속장소명      2
유예시간     632
dtype: int64

In [3]:
df2 = df2.fillna(0)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 894 entries, 0 to 893
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   단속장소명   894 non-null    object 
 1   유예시간    894 non-null    float64
dtypes: float64(1), object(1)
memory usage: 14.1+ KB


In [6]:
df1.isnull().sum()

단속장소명    2
유예시간X    0
dtype: int64

In [4]:
df1 = df1.fillna(0)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 894 entries, 0 to 893
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   단속장소명   894 non-null    object
 1   유예시간X   894 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 14.1+ KB


In [5]:
import numpy as np
print('유예시간아닌시간대의 적발평균:', df1['유예시간X'].mean())

유예시간아닌시간대의 적발평균: 889.423937360179


In [6]:
print('유예시간대의 적발평균:', df2['유예시간'].mean())

유예시간대의 적발평균: 27.10290827740492


In [45]:
df1.head(5)

,단속장소명,유예시간X
0,BYC 주변,108
1,CGV고사점,4271
2,CGV고사점CCTV,5473
3,E마트,7067
4,GS25공구거리점부근,457


In [46]:
df2.head(5)

,단속장소명,유예시간
0,BYC 주변,1.0
1,CGV고사점,0.0
2,CGV고사점CCTV,0.0
3,E마트,1395.0
4,GS25공구거리점부근,0.0


In [47]:
print(df1.info())
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 894 entries, 0 to 893
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   단속장소명   894 non-null    object
 1   유예시간X   894 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 14.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 894 entries, 0 to 893
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   단속장소명   894 non-null    object 
 1   유예시간    894 non-null    float64
dtypes: float64(1), object(1)
memory usage: 14.1+ KB
None


### 독립표본 T검정
#### H0: 유예시간과 유예시간이 아닌 시간대의 적발 평균은 같다
#### H1: 차이가 있다

In [7]:
from scipy.stats import shapiro
#정규성 검정
print(shapiro(df2['유예시간'])) #정규성 만족 
print(shapiro(df1['유예시간X'])) #정규성 만족 

ShapiroResult(statistic=0.19287186861038208, pvalue=0.0)
ShapiroResult(statistic=0.5255110859870911, pvalue=1.233142648605839e-43)


In [8]:
from scipy.stats import levene, ttest_ind
print(levene(df1['유예시간X'], df2['유예시간'])) #등분산성 만족X

LeveneResult(statistic=206.51793590326156, pvalue=2.1380805554713346e-44)


In [9]:
from scipy.stats import levene, ttest_ind
ttest_ind(df1['유예시간X'], df2['유예시간'],  equal_var=False) #분산이 다르므로 

Ttest_indResult(statistic=17.743183135052075, pvalue=1.0504416278214761e-60)

#### 두 데이터 모두 정규성을 만족하지만, 등분산성 만족X
#### 등분산성을 만족하지 않는 독립표본 T검정 결과,
#### 유의수준 0.05 하에서 귀무가설을 기각함. 즉, 두 시간대의 적발횟수 평균에는 차이가 있다.

In [5]:
x = df2['유예시간']

In [9]:
df2.head(5)

,단속장소명,유예시간
0,BYC 주변,1.0
1,CGV고사점,0.0
2,CGV고사점CCTV,0.0
3,E마트,1395.0
4,GS25공구거리점부근,0.0


In [10]:
df2.mean()

유예시간    27.102908
dtype: float64

In [24]:
from statsmodels.stats.diagnostic import kstest_normal
kstest_normal(x, dist='norm')

(0.41991620104027105, 0.0009999999999998899)

### 가설검정
#### H0: 유예시간의 적발 평균은 0이다
#### H1: 유예시간의 적발 평균은 0이 아니다

In [6]:
from scipy.stats import shapiro #정규성 검정
print(shapiro(x)) #정규성 불만족 

ShapiroResult(statistic=0.19287186861038208, pvalue=0.0)


#### p-value: 0.0이므로 정규성 불만족
#### 따라서 모수검정 불가능하고, 비모수 검정 시행

#### H0: 유예시간의 적발 중위수는 0이다
#### H0: 0이 아니다

In [18]:
from scipy.stats import wilcoxon
print('윌콕슨 중위수검정:',wilcoxon(x))

윌콕슨 중위수검정: WilcoxonResult(statistic=0.0, pvalue=8.575371758642613e-45)


#### 검정 결과, H0 REJECT 즉, 유예시간의 적발 중위수는 0이 아니다.

In [14]:
from statsmodels.sandbox.stats.runs import Runs
Runs(df2['유예시간']).runs_test()

(94.13814879361483, 0.0)

In [10]:
from scipy.stats import mannwhitneyu

mannwhitneyu(df1['유예시간X'], df2['유예시간'])

MannwhitneyuResult(statistic=16831.0, pvalue=4.6779366881109285e-282)

In [ ]:
df2.head(5)

In [9]:
df3 = []
df3 = df1['유예시간X']+df2['유예시간']
df3 = pd.DataFrame([df3])

In [10]:
df3 = df3.transpose()

In [11]:
df3 = df3.rename(columns={0:'적발횟수'})

In [12]:
y = df3['적발횟수']

In [19]:
df3.median()

적발횟수    405.5
dtype: float64